# SA1 2021 Census income dataset wrangling

The aim of this notebook is to prepare 2021 census income dataset with these attributes:<br>
-SA1_CODE_2021 <br>
-Median_tot_hhd_inc_weekly <br>
-Tot_Tot <br>
These attributes can be found in two different 2021 census datasets at [Census GeoPackages](https://www.abs.gov.au/census/find-census-data/geopackages?release=2021&geography=AUST&table=G01&gda=GDA2020): <br>
-G02 selected medians and averages <br>
-G33 - Total household income (weekly) by household composition <br>
We combine G02 and G33, then add sa1 geometry boundary to this new dataset. We call this new dataset 'sa1_income_wrangled_2021.geojson'.

In [1]:
import pandas as pd
import geopandas as gpd

In [6]:
df_G02 = pd.read_csv('G02_GDA94_SA1_2021.csv')

G02 dataset contains 'Median_tot_hhd_inc_weekly', but it doesn't provide information about the size of the population.

In [7]:
df_G02

,id,SA1_CODE_2021,SA1_NAME_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size,AREA_ALBERS_SQKM
0,1,10102100701,10102100701,51,1613,689,210,1875,0.7,1583,2.3,362.8727
1,2,10102100702,10102100702,54,1622,818,270,2266,0.7,1774,2.3,229.7459
2,3,10102100703,10102100703,41,1625,834,350,1892,0.8,1542,2.4,2.3910
3,4,10102100704,10102100704,57,1603,732,300,1451,0.7,1182,2.0,1.2816
4,5,10102100705,10102100705,47,1855,779,385,2095,0.7,1468,2.1,1.1978
...,...,...,...,...,...,...,...,...,...,...,...,...
61839,61840,90104100406,90104100406,0,0,0,0,0,0.0,0,0.0,4.9544
61840,61841,90104100407,90104100407,50,1200,675,255,1625,0.8,1179,2.3,10.5956
61841,61842,90104100408,90104100408,0,0,0,0,0,0.0,0,0.0,2.3801
61842,61843,99797979993,99797979993,0,0,0,0,0,0.0,0,0.0,NaN


In [8]:
df_G33 = pd.read_csv('G33_GDA94 _SA1_2021.csv')

G33 contains population count with different income categories.

In [9]:
df_G33

,id,SA1_CODE_2021,SA1_NAME_2021,Neg_Nil_inc_fam_households,Neg_Nil_inc_Non_fam_househlds,Negative_Nil_income_Tot,HI_1_149_Family_households,HI_1_149_Non_family_hseholds,HI_1_149_Tot,HI_150_299_Family_households,...,Part_inc_stated_Fam_household,Part_inc_std_Non_fam_hhds,Partial_income_stated_Tot,All_incomes_ns_Famly_househld,All_inc_ns_Non_famly_househld,All_incomes_not_stated_Tot,Tot_Family_households,Tot_Non_family_households,Tot_Tot,AREA_ALBERS_SQKM
0,1,10102100701,10102100701,0,0,0,0,0,0,0,...,3,0,3,0,0,0,83,42,119,362.8727
1,2,10102100702,10102100702,0,0,0,0,5,5,0,...,15,0,15,4,0,9,87,31,121,229.7459
2,3,10102100703,10102100703,0,0,0,4,0,4,3,...,6,0,7,4,4,7,120,63,187,2.3910
3,4,10102100704,10102100704,0,0,0,0,4,4,0,...,12,0,10,0,4,4,125,91,220,1.2816
4,5,10102100705,10102100705,0,0,0,0,0,0,0,...,9,0,9,0,0,8,111,64,172,1.1978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61839,61840,90104100406,90104100406,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.9544
61840,61841,90104100407,90104100407,0,4,4,0,0,0,0,...,9,0,9,0,0,0,110,53,168,10.5956
61841,61842,90104100408,90104100408,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.3801
61842,61843,99797979993,99797979993,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


Select relevant columns from G33.

In [10]:
df_G33_select = df_G33[["id", "SA1_CODE_2021","Tot_Family_households",\
                                      "Tot_Non_family_households", "Tot_Tot"]]

In [11]:
left = df_G02.set_index('SA1_CODE_2021')

In [12]:
left

,id,SA1_NAME_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size,AREA_ALBERS_SQKM
SA1_CODE_2021,,,,,,,,,,,
10102100701,1,10102100701,51,1613,689,210,1875,0.7,1583,2.3,362.8727
10102100702,2,10102100702,54,1622,818,270,2266,0.7,1774,2.3,229.7459
10102100703,3,10102100703,41,1625,834,350,1892,0.8,1542,2.4,2.3910
10102100704,4,10102100704,57,1603,732,300,1451,0.7,1182,2.0,1.2816
10102100705,5,10102100705,47,1855,779,385,2095,0.7,1468,2.1,1.1978
...,...,...,...,...,...,...,...,...,...,...,...
90104100406,61840,90104100406,0,0,0,0,0,0.0,0,0.0,4.9544
90104100407,61841,90104100407,50,1200,675,255,1625,0.8,1179,2.3,10.5956
90104100408,61842,90104100408,0,0,0,0,0,0.0,0,0.0,2.3801


In [13]:
right = df_G33_select.set_index('SA1_CODE_2021')

In [14]:
right

,id,Tot_Family_households,Tot_Non_family_households,Tot_Tot
SA1_CODE_2021,,,,
10102100701,1,83,42,119
10102100702,2,87,31,121
10102100703,3,120,63,187
10102100704,4,125,91,220
10102100705,5,111,64,172
...,...,...,...,...
90104100406,61840,0,0,0
90104100407,61841,110,53,168
90104100408,61842,0,0,0


Join relevant columns of G02 and G33 into one dataset.

In [15]:
result = left.join(right, lsuffix='_left', rsuffix='_right', how = 'inner')

In [16]:
result

,id_left,SA1_NAME_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size,AREA_ALBERS_SQKM,id_right,Tot_Family_households,Tot_Non_family_households,Tot_Tot
SA1_CODE_2021,,,,,,,,,,,,,,,
10102100701,1,10102100701,51,1613,689,210,1875,0.7,1583,2.3,362.8727,1,83,42,119
10102100702,2,10102100702,54,1622,818,270,2266,0.7,1774,2.3,229.7459,2,87,31,121
10102100703,3,10102100703,41,1625,834,350,1892,0.8,1542,2.4,2.3910,3,120,63,187
10102100704,4,10102100704,57,1603,732,300,1451,0.7,1182,2.0,1.2816,4,125,91,220
10102100705,5,10102100705,47,1855,779,385,2095,0.7,1468,2.1,1.1978,5,111,64,172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90104100406,61840,90104100406,0,0,0,0,0,0.0,0,0.0,4.9544,61840,0,0,0
90104100407,61841,90104100407,50,1200,675,255,1625,0.8,1179,2.3,10.5956,61841,110,53,168
90104100408,61842,90104100408,0,0,0,0,0,0.0,0,0.0,2.3801,61842,0,0,0


Downloads for GDA94 digital boundary files-Statistical Areas Level 1 - 2021 - Shapefile
https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files

In [28]:
#Note: need to upload all files in zip file of 'SA1_2021_AUST_SHP_GDA94 (1).zip'.
SA1_shape = gpd.read_file("SA1_2021_AUST_GDA94.shp")


In [29]:
SA1_shape

,SA1_CODE21,CHG_FLAG21,CHG_LBL21,SA2_CODE21,SA2_NAME21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,10102100701,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,362.8727,http://linked.data.gov.au/dataset/asgsed3/SA1/...,"POLYGON ((149.89110 -35.08990, 149.89181 -35.0..."
1,10102100702,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,229.7459,http://linked.data.gov.au/dataset/asgsed3/SA1/...,"POLYGON ((149.73421 -35.36758, 149.74475 -35.3..."
2,10102100703,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,2.3910,http://linked.data.gov.au/dataset/asgsed3/SA1/...,"POLYGON ((149.77998 -35.44067, 149.77996 -35.4..."
3,10102100704,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,1.2816,http://linked.data.gov.au/dataset/asgsed3/SA1/...,"POLYGON ((149.79744 -35.44149, 149.79778 -35.4..."
4,10102100705,0,No change,101021007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,1.1978,http://linked.data.gov.au/dataset/asgsed3/SA1/...,"POLYGON ((149.79498 -35.44902, 149.79517 -35.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61840,90104100407,0,No change,901041004,Norfolk Island,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,10.5956,http://linked.data.gov.au/dataset/asgsed3/SA1/...,"MULTIPOLYGON (((167.91661 -29.00961, 167.91659..."
61841,90104100408,0,No change,901041004,Norfolk Island,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,2.3801,http://linked.data.gov.au/dataset/asgsed3/SA1/...,"MULTIPOLYGON (((167.96325 -29.07213, 167.96326..."
61842,99797979993,0,No change,997979799,Migratory - Offshore - Shipping (OT),99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA1/...,None
61843,99999949999,0,No change,999999499,No usual address (OT),99999,No usual address (OT),999,No usual address (OT),99499,No usual address (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA1/...,None


In [31]:
SA1_geometry = SA1_shape[['SA1_CODE21', 'geometry']]

In [32]:
#rename columns
SA1_geometry.columns = ['SA1_CODE_2021', 'geometry']

In [33]:
SA1_geometry

,SA1_CODE_2021,geometry
0,10102100701,"POLYGON ((149.89110 -35.08990, 149.89181 -35.0..."
1,10102100702,"POLYGON ((149.73421 -35.36758, 149.74475 -35.3..."
2,10102100703,"POLYGON ((149.77998 -35.44067, 149.77996 -35.4..."
3,10102100704,"POLYGON ((149.79744 -35.44149, 149.79778 -35.4..."
4,10102100705,"POLYGON ((149.79498 -35.44902, 149.79517 -35.4..."
...,...,...
61840,90104100407,"MULTIPOLYGON (((167.91661 -29.00961, 167.91659..."
61841,90104100408,"MULTIPOLYGON (((167.96325 -29.07213, 167.96326..."
61842,99797979993,None
61843,99999949999,None


In [34]:
right = SA1_geometry.set_index('SA1_CODE_2021')

In [35]:
right

,geometry
SA1_CODE_2021,
10102100701,"POLYGON ((149.89110 -35.08990, 149.89181 -35.0..."
10102100702,"POLYGON ((149.73421 -35.36758, 149.74475 -35.3..."
10102100703,"POLYGON ((149.77998 -35.44067, 149.77996 -35.4..."
10102100704,"POLYGON ((149.79744 -35.44149, 149.79778 -35.4..."
10102100705,"POLYGON ((149.79498 -35.44902, 149.79517 -35.4..."
...,...
90104100407,"MULTIPOLYGON (((167.91661 -29.00961, 167.91659..."
90104100408,"MULTIPOLYGON (((167.96325 -29.07213, 167.96326..."
99797979993,None


In [36]:
income_2021 = result.join(right, lsuffix='_left', rsuffix='_right', how = 'inner')

In [38]:
income_2021

,id_left,SA1_NAME_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size,AREA_ALBERS_SQKM,id_right,Tot_Family_households,Tot_Non_family_households,Tot_Tot,geometry
SA1_CODE_2021,,,,,,,,,,,,,,,,


In [39]:
result

,id_left,SA1_NAME_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size,AREA_ALBERS_SQKM,id_right,Tot_Family_households,Tot_Non_family_households,Tot_Tot
SA1_CODE_2021,,,,,,,,,,,,,,,
10102100701,1,10102100701,51,1613,689,210,1875,0.7,1583,2.3,362.8727,1,83,42,119
10102100702,2,10102100702,54,1622,818,270,2266,0.7,1774,2.3,229.7459,2,87,31,121
10102100703,3,10102100703,41,1625,834,350,1892,0.8,1542,2.4,2.3910,3,120,63,187
10102100704,4,10102100704,57,1603,732,300,1451,0.7,1182,2.0,1.2816,4,125,91,220
10102100705,5,10102100705,47,1855,779,385,2095,0.7,1468,2.1,1.1978,5,111,64,172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90104100406,61840,90104100406,0,0,0,0,0,0.0,0,0.0,4.9544,61840,0,0,0
90104100407,61841,90104100407,50,1200,675,255,1625,0.8,1179,2.3,10.5956,61841,110,53,168
90104100408,61842,90104100408,0,0,0,0,0,0.0,0,0.0,2.3801,61842,0,0,0


In [40]:
left = result

In [41]:
left

,id_left,SA1_NAME_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size,AREA_ALBERS_SQKM,id_right,Tot_Family_households,Tot_Non_family_households,Tot_Tot
SA1_CODE_2021,,,,,,,,,,,,,,,
10102100701,1,10102100701,51,1613,689,210,1875,0.7,1583,2.3,362.8727,1,83,42,119
10102100702,2,10102100702,54,1622,818,270,2266,0.7,1774,2.3,229.7459,2,87,31,121
10102100703,3,10102100703,41,1625,834,350,1892,0.8,1542,2.4,2.3910,3,120,63,187
10102100704,4,10102100704,57,1603,732,300,1451,0.7,1182,2.0,1.2816,4,125,91,220
10102100705,5,10102100705,47,1855,779,385,2095,0.7,1468,2.1,1.1978,5,111,64,172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90104100406,61840,90104100406,0,0,0,0,0,0.0,0,0.0,4.9544,61840,0,0,0
90104100407,61841,90104100407,50,1200,675,255,1625,0.8,1179,2.3,10.5956,61841,110,53,168
90104100408,61842,90104100408,0,0,0,0,0,0.0,0,0.0,2.3801,61842,0,0,0


In [42]:
right

,geometry
SA1_CODE_2021,
10102100701,"POLYGON ((149.89110 -35.08990, 149.89181 -35.0..."
10102100702,"POLYGON ((149.73421 -35.36758, 149.74475 -35.3..."
10102100703,"POLYGON ((149.77998 -35.44067, 149.77996 -35.4..."
10102100704,"POLYGON ((149.79744 -35.44149, 149.79778 -35.4..."
10102100705,"POLYGON ((149.79498 -35.44902, 149.79517 -35.4..."
...,...
90104100407,"MULTIPOLYGON (((167.91661 -29.00961, 167.91659..."
90104100408,"MULTIPOLYGON (((167.96325 -29.07213, 167.96326..."
99797979993,None


In [43]:
income_2021 = left.join(right, lsuffix='_left', rsuffix='_right', how = 'inner')

In [44]:
income_2021

,id_left,SA1_NAME_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size,AREA_ALBERS_SQKM,id_right,Tot_Family_households,Tot_Non_family_households,Tot_Tot,geometry
SA1_CODE_2021,,,,,,,,,,,,,,,,


Here the two datasets can't be joined like the previous case, is because these two datasets have different format: one is csv, the other is shapefile. We have to find the right way to join these two different formatting files. <br>
Resource like this: https://stackoverflow.com/questions/69817876/merge-a-csv-and-shapefile-with-geopandas <br>
`gdf = gpd.GeoDataFrame(df.merge(gdf2, on="iso_3166_2"))`

In [45]:
gdf = gpd.GeoDataFrame(left.merge(right, on = 'SA1_CODE_2021'))

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

Now we have a problem, as the error message says, 'You are trying to merge on int64 and object columns.' We need to find out if the 'SA1_CODE_2021' from left and right are different data type, one 'int64' and the other one 'object'. 

Data type of the left:

In [47]:
left.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61844 entries, 10102100701 to 99999949999
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id_left                        61844 non-null  int64  
 1   SA1_NAME_2021                  61844 non-null  int64  
 2   Median_age_persons             61844 non-null  int64  
 3   Median_mortgage_repay_monthly  61844 non-null  int64  
 4   Median_tot_prsnl_inc_weekly    61844 non-null  int64  
 5   Median_rent_weekly             61844 non-null  int64  
 6   Median_tot_fam_inc_weekly      61844 non-null  int64  
 7   Average_num_psns_per_bedroom   61844 non-null  float64
 8   Median_tot_hhd_inc_weekly      61844 non-null  int64  
 9   Average_household_size         61844 non-null  float64
 10  AREA_ALBERS_SQKM               61811 non-null  float64
 11  id_right                       61844 non-null  int64  
 12  Tot_Family_households         

Data type of the right:

In [48]:
right.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 61845 entries, 10102100701 to ZZZZZZZZZZZ
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  61811 non-null  geometry
dtypes: geometry(1)
memory usage: 3.0+ MB


It fails to show the index data type, so we need to reset the index and find out the data type of 'SA1_CODE_2021' on the right. 

In [51]:
right = right.reset_index()

In [52]:
right.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 61845 entries, 0 to 61844
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   SA1_CODE_2021  61845 non-null  object  
 1   geometry       61811 non-null  geometry
dtypes: geometry(1), object(1)
memory usage: 966.5+ KB


In [53]:
left = left.reset_index()

In [54]:
left

,SA1_CODE_2021,id_left,SA1_NAME_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size,AREA_ALBERS_SQKM,id_right,Tot_Family_households,Tot_Non_family_households,Tot_Tot
0,10102100701,1,10102100701,51,1613,689,210,1875,0.7,1583,2.3,362.8727,1,83,42,119
1,10102100702,2,10102100702,54,1622,818,270,2266,0.7,1774,2.3,229.7459,2,87,31,121
2,10102100703,3,10102100703,41,1625,834,350,1892,0.8,1542,2.4,2.3910,3,120,63,187
3,10102100704,4,10102100704,57,1603,732,300,1451,0.7,1182,2.0,1.2816,4,125,91,220
4,10102100705,5,10102100705,47,1855,779,385,2095,0.7,1468,2.1,1.1978,5,111,64,172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61839,90104100406,61840,90104100406,0,0,0,0,0,0.0,0,0.0,4.9544,61840,0,0,0
61840,90104100407,61841,90104100407,50,1200,675,255,1625,0.8,1179,2.3,10.5956,61841,110,53,168
61841,90104100408,61842,90104100408,0,0,0,0,0,0.0,0,0.0,2.3801,61842,0,0,0
61842,99797979993,61843,99797979993,0,0,0,0,0,0.0,0,0.0,NaN,61843,0,0,0


In [55]:
left.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61844 entries, 0 to 61843
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SA1_CODE_2021                  61844 non-null  int64  
 1   id_left                        61844 non-null  int64  
 2   SA1_NAME_2021                  61844 non-null  int64  
 3   Median_age_persons             61844 non-null  int64  
 4   Median_mortgage_repay_monthly  61844 non-null  int64  
 5   Median_tot_prsnl_inc_weekly    61844 non-null  int64  
 6   Median_rent_weekly             61844 non-null  int64  
 7   Median_tot_fam_inc_weekly      61844 non-null  int64  
 8   Average_num_psns_per_bedroom   61844 non-null  float64
 9   Median_tot_hhd_inc_weekly      61844 non-null  int64  
 10  Average_household_size         61844 non-null  float64
 11  AREA_ALBERS_SQKM               61811 non-null  float64
 12  id_right                       61844 non-null 

Now we confirmed the problem. 'left' 'SA1_CODE_2021' is int64. 'right' 'SA1_CODE_2021' is object. That is why it says 'You are trying to merge on int64 and object columns.' <br>
Found this: https://www.statology.org/you-are-trying-to-merge-on-object-and-int64-columns/
Example: <br>
#convert year variable in df2 to integer<br>
`df2['year']=df2['year'].astype(int)`

In [56]:
#convert 'SA1_CODE_2021' variable in right to integer
right['SA1_CODE_2021'] = right['SA1_CODE_2021'].astype(int)

ValueError: invalid literal for int() with base 10: 'ZZZZZZZZZZZ'

'ZZZZZZZZZZZ' is from the last row of right. We need to drop the last row. i.e. `df.drop([0, 1])`

In [57]:
right = right.drop([61844])

In [58]:
right

,SA1_CODE_2021,geometry
0,10102100701,"POLYGON ((149.89110 -35.08990, 149.89181 -35.0..."
1,10102100702,"POLYGON ((149.73421 -35.36758, 149.74475 -35.3..."
2,10102100703,"POLYGON ((149.77998 -35.44067, 149.77996 -35.4..."
3,10102100704,"POLYGON ((149.79744 -35.44149, 149.79778 -35.4..."
4,10102100705,"POLYGON ((149.79498 -35.44902, 149.79517 -35.4..."
...,...,...
61839,90104100406,"MULTIPOLYGON (((167.95810 -29.00470, 167.95808..."
61840,90104100407,"MULTIPOLYGON (((167.91661 -29.00961, 167.91659..."
61841,90104100408,"MULTIPOLYGON (((167.96325 -29.07213, 167.96326..."
61842,99797979993,None


In [59]:
#now, try again.
#convert 'SA1_CODE_2021' variable in right to integer
right['SA1_CODE_2021'] = right['SA1_CODE_2021'].astype(int)

In [60]:
income = left.merge(right, how='left', on='SA1_CODE_2021')

In [61]:
income

,SA1_CODE_2021,id_left,SA1_NAME_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size,AREA_ALBERS_SQKM,id_right,Tot_Family_households,Tot_Non_family_households,Tot_Tot,geometry
0,10102100701,1,10102100701,51,1613,689,210,1875,0.7,1583,2.3,362.8727,1,83,42,119,"POLYGON ((149.89110 -35.08990, 149.89181 -35.0..."
1,10102100702,2,10102100702,54,1622,818,270,2266,0.7,1774,2.3,229.7459,2,87,31,121,"POLYGON ((149.73421 -35.36758, 149.74475 -35.3..."
2,10102100703,3,10102100703,41,1625,834,350,1892,0.8,1542,2.4,2.3910,3,120,63,187,"POLYGON ((149.77998 -35.44067, 149.77996 -35.4..."
3,10102100704,4,10102100704,57,1603,732,300,1451,0.7,1182,2.0,1.2816,4,125,91,220,"POLYGON ((149.79744 -35.44149, 149.79778 -35.4..."
4,10102100705,5,10102100705,47,1855,779,385,2095,0.7,1468,2.1,1.1978,5,111,64,172,"POLYGON ((149.79498 -35.44902, 149.79517 -35.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61839,90104100406,61840,90104100406,0,0,0,0,0,0.0,0,0.0,4.9544,61840,0,0,0,"MULTIPOLYGON (((167.95810 -29.00470, 167.95808..."
61840,90104100407,61841,90104100407,50,1200,675,255,1625,0.8,1179,2.3,10.5956,61841,110,53,168,"MULTIPOLYGON (((167.91661 -29.00961, 167.91659..."
61841,90104100408,61842,90104100408,0,0,0,0,0,0.0,0,0.0,2.3801,61842,0,0,0,"MULTIPOLYGON (((167.96325 -29.07213, 167.96326..."
61842,99797979993,61843,99797979993,0,0,0,0,0,0.0,0,0.0,NaN,61843,0,0,0,None


Now we finally added the geometry into the income dataset. Now it is time to recognise 'geometry' as a geospatial column.
`geopandas.GeoDataFrame.set_geometry`

In [62]:
income_gdf = income.set_geometry("geometry")

In [63]:
income_gdf

,SA1_CODE_2021,id_left,SA1_NAME_2021,Median_age_persons,Median_mortgage_repay_monthly,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_tot_fam_inc_weekly,Average_num_psns_per_bedroom,Median_tot_hhd_inc_weekly,Average_household_size,AREA_ALBERS_SQKM,id_right,Tot_Family_households,Tot_Non_family_households,Tot_Tot,geometry
0,10102100701,1,10102100701,51,1613,689,210,1875,0.7,1583,2.3,362.8727,1,83,42,119,"POLYGON ((149.89110 -35.08990, 149.89181 -35.0..."
1,10102100702,2,10102100702,54,1622,818,270,2266,0.7,1774,2.3,229.7459,2,87,31,121,"POLYGON ((149.73421 -35.36758, 149.74475 -35.3..."
2,10102100703,3,10102100703,41,1625,834,350,1892,0.8,1542,2.4,2.3910,3,120,63,187,"POLYGON ((149.77998 -35.44067, 149.77996 -35.4..."
3,10102100704,4,10102100704,57,1603,732,300,1451,0.7,1182,2.0,1.2816,4,125,91,220,"POLYGON ((149.79744 -35.44149, 149.79778 -35.4..."
4,10102100705,5,10102100705,47,1855,779,385,2095,0.7,1468,2.1,1.1978,5,111,64,172,"POLYGON ((149.79498 -35.44902, 149.79517 -35.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61839,90104100406,61840,90104100406,0,0,0,0,0,0.0,0,0.0,4.9544,61840,0,0,0,"MULTIPOLYGON (((167.95810 -29.00470, 167.95808..."
61840,90104100407,61841,90104100407,50,1200,675,255,1625,0.8,1179,2.3,10.5956,61841,110,53,168,"MULTIPOLYGON (((167.91661 -29.00961, 167.91659..."
61841,90104100408,61842,90104100408,0,0,0,0,0,0.0,0,0.0,2.3801,61842,0,0,0,"MULTIPOLYGON (((167.96325 -29.07213, 167.96326..."
61842,99797979993,61843,99797979993,0,0,0,0,0,0.0,0,0.0,NaN,61843,0,0,0,None


In [64]:
income_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 61844 entries, 0 to 61843
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   SA1_CODE_2021                  61844 non-null  int64   
 1   id_left                        61844 non-null  int64   
 2   SA1_NAME_2021                  61844 non-null  int64   
 3   Median_age_persons             61844 non-null  int64   
 4   Median_mortgage_repay_monthly  61844 non-null  int64   
 5   Median_tot_prsnl_inc_weekly    61844 non-null  int64   
 6   Median_rent_weekly             61844 non-null  int64   
 7   Median_tot_fam_inc_weekly      61844 non-null  int64   
 8   Average_num_psns_per_bedroom   61844 non-null  float64 
 9   Median_tot_hhd_inc_weekly      61844 non-null  int64   
 10  Average_household_size         61844 non-null  float64 
 11  AREA_ALBERS_SQKM               61811 non-null  float64 
 12  id_right                

In [65]:
income_gdf.to_file('sa1_income_wrangled_2021.geojson', driver='GeoJSON')

Done!